In [18]:
import numpy as np
import os
import xml.etree.ElementTree as ET
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import train_test_split

In [2]:
projectDir = r'C:\Users\Viktor\Skola\embedded\embedded_project'

homeDir = os.path.join(projectDir,'keras_Realtime_Multi-Person_Pose_Estimation-master')
xmlDir = os.path.join(projectDir, 'Stanford40_XMLAnnotations\XMLAnnotations')

#homeDir = r'C:\Users\Viktor\Skola\embedded\embedded_project\keras_Realtime_Multi-Person_Pose_Estimation-master'
#xmlDir = r'C:\Users\Viktor\Skola\embedded\embedded_project\Stanford40_XMLAnnotations\XMLAnnotations'

In [3]:
x = np.load(os.path.join(homeDir,'partCoords.npy'))
print(x.shape)

(9532, 18, 2)


In [4]:
#np.set_printoptions(suppress=True)

#x10 = x[:10]

#print(x10)

#x10_norm = np.zeros(x10.shape)

for i in range(len(x)):
    #print(x10[i])
    xmax, ymax = np.nanmax(x[i], axis=0)
    xmin, ymin = np.nanmin(x[i], axis=0)
    
    #if np.isnan(xmax):
    #    print(x[i])
    #print(i)
    #print(xmin)
    #print(xmax)
    #print(ymin)
    #print(ymax)
    
    x[i][:,0] -= xmin
    x[i][:,0] /= (xmax - xmin)
    
    x[i][:,1] -= ymin
    x[i][:,1] /= (ymax - ymin)
    
    #print(x[i])
#print(x10_norm)
#print(x[1297])

<ipython-input-4-733908754da7>:11: RuntimeWarning: All-NaN slice encountered
  xmax, ymax = np.nanmax(x[i], axis=0)
<ipython-input-4-733908754da7>:12: RuntimeWarning: All-NaN slice encountered
  xmin, ymin = np.nanmin(x[i], axis=0)


In [5]:
x[np.isnan(x)] = 0

In [6]:
print(x)

[[[0.54545455 0.04210526]
  [0.4171123  0.29824561]
  [0.07486631 0.34035088]
  ...
  [0.55080214 0.        ]
  [0.25668449 0.08070175]
  [0.         0.        ]]

 [[0.46       0.05291005]
  [0.445      0.47619048]
  [0.1        0.49206349]
  ...
  [0.525      0.00529101]
  [0.265      0.1005291 ]
  [0.6        0.11640212]]

 [[0.39344262 0.11111111]
  [0.78688525 0.51111111]
  [1.         0.43333333]
  ...
  [0.45901639 0.01111111]
  [0.         0.        ]
  [0.74590164 0.        ]]

 ...

 [[0.78723404 0.29166667]
  [0.56382979 0.48611111]
  [0.24468085 0.40277778]
  ...
  [0.84574468 0.22916667]
  [0.57978723 0.        ]
  [0.         0.        ]]

 [[0.53784861 0.18315018]
  [0.60756972 0.27838828]
  [0.35258964 0.23076923]
  ...
  [0.62749004 0.10989011]
  [0.46414343 0.        ]
  [0.80278884 0.        ]]

 [[0.24418605 0.2755102 ]
  [0.70930233 0.34693878]
  [0.38372093 0.32653061]
  ...
  [0.34883721 0.15306122]
  [0.         0.        ]
  [0.80232558 0.        ]]]


In [7]:
print(x[0].shape)

(18, 2)


In [8]:
os.chdir(homeDir)

jpgOrder = open('JPG_Order.txt', 'r')
lines = jpgOrder.readlines()

y_string = []

os.chdir(xmlDir)

for i in range(len(lines)):
    imgFileName = lines[i].strip()
    fileName = imgFileName.split(".")[0]
    xmlFileName = fileName + ".xml"
    
    root = ET.parse(xmlFileName).getroot()
    action = root.find('object/action').text
    
    y_string.append(action)
    
    print(str(i) + '/' + str(len(lines)),end='\r')

#print(y_string)

In [9]:
(allUniqueClasses,y_int) = np.unique(y_string, return_inverse=True)

print(y_int)

y_cat = tf.keras.utils.to_categorical(y_int, len(allUniqueClasses))

print(y_cat)

[ 0  0  0 ... 39 39 39]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y_cat, test_size=0.2, random_state=42)

In [10]:
def defineModel(nrOfClasses):
    model = Sequential()
    model.add(keras.Input(shape=(18,2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    
    model.add(Dense(nrOfClasses, activation='softmax'))
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [11]:
model = defineModel(40)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 36)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               9472      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 40)                10280     
Total params: 85,544
Trainable params: 85,544
Non-trainable params: 0
_________________________________________________________________


In [12]:
#x = x[np.newaxis,...]

In [13]:
#y_cat = y_cat[np.newaxis,...]

In [20]:
model.fit(x_train,y_train,epochs=20,verbose=1)

Epoch 1/20
239/239 [==============================] - 0s 1ms/step - loss: 1.2695 - accuracy: 0.6110
Epoch 2/20
239/239 [==============================] - 0s 945us/step - loss: 1.2322 - accuracy: 0.6211
Epoch 3/20
239/239 [==============================] - 0s 1ms/step - loss: 1.1982 - accuracy: 0.6366
Epoch 4/20
239/239 [==============================] - 0s 1ms/step - loss: 1.1688 - accuracy: 0.6472
Epoch 5/20
239/239 [==============================] - 0s 1ms/step - loss: 1.1375 - accuracy: 0.6548
Epoch 6/20
239/239 [==============================] - 0s 1ms/step - loss: 1.1057 - accuracy: 0.6657
Epoch 7/20
239/239 [==============================] - 0s 960us/step - loss: 1.0838 - accuracy: 0.6682
Epoch 8/20
239/239 [==============================] - 0s 955us/step - loss: 1.0556 - accuracy: 0.6843
Epoch 9/20
239/239 [==============================] - 0s 984us/step - loss: 1.0356 - accuracy: 0.6830
Epoch 10/20
239/239 [==============================] - 0s 960us/step - loss: 1.0118 - accura

In [21]:
model.evaluate(x_test,y_test)

60/60 [==============================] - 0s 781us/step - loss: 1.8813 - accuracy: 0.4646


[1.8812940120697021, 0.46460407972335815]